In [24]:
import numpy as np
from pathlib import Path
from dataloader import loadForumData

dir = "./data/hackerForums/"

df = loadForumData(dir)

# No clue why these Null values are here, should be filtered in the dataloader already
df = df[~df["PostContent"].isnull()]

c:\Users\Rabjho\OneDrive - Aarhus universitet\Introduction to NLP and Generative AI\AU-Introduction-NLP\dataloader.py:13: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
c:\Users\Rabjho\OneDrive - Aarhus universitet\Introduction to NLP and Generative AI\AU-Introduction-NLP\dataloader.py:13: DtypeWarning: Columns (17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [25]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

ner = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

c:\Users\Rabjho\OneDrive - Aarhus universitet\Introduction to NLP and Generative AI\AU-Introduction-NLP\.venv\lib\site-packages\transformers\pipelines\token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [32]:
# Make a subset and check how fast it is
temp = df.head(100)
temp["entities"] = temp["PostContent"].apply(lambda x: ner(x))

temp.to_csv("processedDataTest.csv", index=False)

# df["entities"] = df["PostContent"].apply(lambda x: ner(x))

# df.to_csv("processedData.csv", index=False)

C:\Users\Rabjho\AppData\Local\Temp\ipykernel_13372\1278343691.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["entities"] = temp["PostContent"].apply(lambda x: ner(x))
